In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm

import numpy as np
from itertools import chain, combinations, combinations_with_replacement, product
from scipy.special import erfc
import pickle as pk
from time import time
import math

# utility functions
def save_pk(data, file):
    with open(file, 'wb') as f:
        pk.dump(data, f)


27
[-4 -3 -2 -1  0  1  2  3  4]
232
(232, 3)


In [96]:
def generate_vectors(vector_set, rcut, nimgs=10, sphere=True):
    
    nimgs = np.ceil(rcut*reciprocal_height[0] + 1.1).astype(int)
    img_range = np.arange(-nimgs, nimgs+1)
    print(img_range)
    # get all possible combinations of the basis vectors
   # get all possible combinations of the basis vectors
    img_sets = list(product(*[img_range, img_range, img_range]))
    img_sets = np.concatenate([np.array(x)[None, :, None] for x in img_sets if not np.sum(np.abs(x)) == 0], axis=0)
    # print(img_sets)

    imgs = np.sum(img_sets * basis_cell, axis=1)
    x = np.unique(imgs, axis=0)
    print(len(imgs), len(x))
    
    v = np.split(basis_cell, 3, axis=0)
    z = np.zeros_like(v[0])
    # print(v.shape, z.shape)
    vecs = product(*[[-v[0], z, v[0]],[-v[1], z, v[1]], [-v[2], z, v[2]]] )
    vecs = np.array(list(vecs)).squeeze().sum(-2)


    # remove the copies
    if sphere:
        # remove outside the radius (so it is a sphere) it doesn't make a difference to the current implementation
        lengths = np.linalg.norm(vecs[None, ...] + imgs[:, None, :], axis=-1)
        mask = np.any(lengths < rcut, axis=1)

#         print('n vectors cut: ', np.sum(mask))
        imgs = imgs[mask]

    return imgs

def compute_volume(v1, v2, v3):
    cross = np.cross(v2, v3, axisa=0, axisb=0)
    box = np.sum(v1 * cross)
    return np.abs(np.squeeze(box))

def vector_sub(v1, v2):
    return np.expand_dims(v1, axis=-2) - np.expand_dims(v2, axis=-3)

def vector_add(v1, v2):
    return np.expand_dims(v1, axis=2) + np.expand_dims(v2, axis=1)

def vector_prod(v1, v2):
    return np.expand_dims(v1, axis=2) * np.expand_dims(v2, axis=1)

def compute_distances(v1, v2):
    inter_vector = vector_sub(v1, v2)
    return np.sqrt(np.sum(inter_vector**2, axis=-1))

def inner(v1, v2):
    return np.sum(v1 * v2, axis=-1)

In [138]:
#https://unlcms.unl.edu/cas/physics/tsymbal/teaching/SSP-927/Section%2001_Crystal%20Structure.pdf

basis_cell = np.array([[-0.5, 0.5, 0.5],
                       [0.5, -0.5, 0.5],
                       [0.5, 0.5, -0.5]])
print(abs(np.linalg.det(basis_cell)))
inv_cell_basis = np.linalg.inv(basis_cell)
center = np.sum(basis_cell, axis=0) / 2.
print('center: ', center)

cv1, cv2, cv3 = np.split(basis_cell, 3, axis=1)
print('primitive vectors: ', '\n', cv1, '\n', cv2, '\n', cv3)
volume = compute_volume(cv1, cv2, cv3)
print('volume', volume)

rv1 = 2 * np.pi * np.cross(cv2.squeeze(), cv3.squeeze()) / volume
rv2 = 2 * np.pi * np.cross(cv3.squeeze(), cv1.squeeze()) / volume
rv3 = 2 * np.pi * np.cross(cv1.squeeze(), cv2.squeeze()) / volume
reciprocal_cell = np.concatenate([x[None, :] for x in (rv1, rv2, rv3)], axis=0)
normed_to1 = reciprocal_vectors / (2 * np.pi)
reciprocal_height = np.linalg.norm(normed_to1, axis=1)


reciprocal_volume = abs(np.linalg.det(reciprocal_vectors))
print('reciprocal volume: ', reciprocal_volume)

print('recipricol vectors: ', '\n', rv1, '\n', rv2, '\n', rv3)

n_walkers = 1
n_el = 3

r_atoms = np.array([[[0.25, 0.25, 0.25]]]).repeat(n_walkers, axis=0) - center
print('r_atom: ', r_atoms)

r_charges = np.array([[float(n_el)]]).repeat(n_walkers, axis=0)

walkers = np.concatenate([r_atoms,], axis=1)
charges = np.concatenate([r_charges,], axis=1)
print(walkers.shape, charges.shape)


0.5
center:  [0.25 0.25 0.25]
primitive vectors:  
 [[-0.5]
 [ 0.5]
 [ 0.5]] 
 [[ 0.5]
 [-0.5]
 [ 0.5]] 
 [[ 0.5]
 [ 0.5]
 [-0.5]]
volume 0.5
reciprocal volume:  496.10042688479695
recipricol vectors:  
 [0.         6.28318531 6.28318531] 
 [6.28318531 0.         6.28318531] 
 [6.28318531 6.28318531 0.        ]
r_atom:  [[[0. 0. 0.]]]
(1, 1, 3) (1, 1)


In [139]:
def cartesian_prod(arrays, out=None):
    arrays = [np.asarray(x) for x in arrays]
    dtype = np.result_type(*arrays)
    nd = len(arrays)
    dims = [nd] + [len(x) for x in arrays]
    out = np.ndarray(dims, dtype, buffer=out)

    shape = [-1] + [1] * nd
    for i, arr in enumerate(arrays):
        out[i] = arr.reshape(shape[:nd-i])

    return out.reshape(nd,-1).T


def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

mesh = [10, 10, 10]
rx = np.fft.fftfreq(mesh[0], 1./mesh[0])
ry = np.fft.fftfreq(mesh[1], 1./mesh[1])
rz = np.fft.fftfreq(mesh[2], 1./mesh[2])

Gvbase = (rx, ry, rz)

x = cartesian_product(*Gvbase)
y = cartesian_prod(Gvbase)
z = np.concatenate([x, y], axis=0)
print(len(np.unique(z, axis=0)))

1000


In [140]:

def create_mesh(mesh):
    mesh = np.copy(mesh)
    mesh_max = np.asarray(np.linalg.norm(basis_cell, axis=1) * 2,
                          dtype=int)  # roughly 2 grids per bohr
    mesh_max[mesh_max<80] = 80
    mesh[mesh>mesh_max] = mesh_max[mesh>mesh_max]
    return mesh





def get_gv_weights(reciprocal_vectors, mesh=[10, 10, 10]):
    mesh = create_mesh(mesh)
    print(mesh)
    rx = np.fft.fftfreq(mesh[0], 1./mesh[0])
    ry = np.fft.fftfreq(mesh[1], 1./mesh[1])
    rz = np.fft.fftfreq(mesh[2], 1./mesh[2])
    
    weights = abs(np.linalg.det(reciprocal_vectors))
    weights *= (1./ (2*np.pi)**3)

    Gvbase = (rx, ry, rz)
    gv = np.dot(cartesian_product(*Gvbase), reciprocal_vectors)
    print(gv.shape)
    return gv, weights

gv, w = get_gv_weights(reciprocal_vectors)

print(gv.shape)
    

[10 10 10]
(1000, 3)
(1000, 3)


In [142]:
def compute_potential(walkers, charges, kappa, rcut, test=False):
    lattice_vectors = generate_vectors(basis_cell, rcut)
    r_vectors, weights = get_gv_weights(reciprocal_vectors)
    r_vectors = generate_vectors(reciprocal_vectors, rcut)
    print(len(lattice_vectors), len(r_vectors))
    # compute the Rs0 term
    e_e_vectors = vector_sub(walkers, walkers)
    q_q = vector_prod(charges, charges)
    e_e_distances = compute_distances(walkers, walkers)
    # is half the value because only the lower half is taken
    e_e_Rs0 = q_q * np.tril(erfc(kappa * e_e_distances) / e_e_distances, k=-1)  

    # compute the Rs > 0 term
    ex_walkers = vector_add(walkers, lattice_vectors[None, ...])
    tmp = walkers[..., None, None, :] - ex_walkers[:, None, ...]
    ex_distances = np.sqrt(np.sum(tmp**2, axis=-1))
    e_e_Rs1 = 0.5 * q_q * np.sum(erfc(kappa * ex_distances) / ex_distances, axis=-1)
    print(e_e_Rs1)

    # compute the constant factor
    self_interaction = 0.5 * q_q * np.diag(np.array([2 * kappa / np.sqrt(np.pi)]).repeat(walkers.shape[1], axis=0))[None, ...]
    
    constant = 0.5 * np.pi * np.sum(charges)**2 / (kappa**2 * volume)

    # compute the reciprocal term reuse the ee vectors
    exp = np.exp(1j * e_e_vectors @ np.transpose(r_vectors))
    r_inner = inner(r_vectors, r_vectors) 
    r_inner[r_inner == 0.] = 1e200
    r_factor = 4 * np.pi * weights * (np.exp(- r_inner / (4 * kappa**2)) / r_inner)[None, None, None, :] * exp
    r_term = 0.5 * q_q * np.real(np.sum(r_factor, axis=-1)) 
    
    print('r_term:', r_term)
    print('self_interation', - self_interaction)
    print('constant ', -constant)
    average_potential = e_e_Rs0 + e_e_Rs1 + r_term
    average_potential -= constant
    average_potential -= self_interaction
    average_potential = np.sum(average_potential, axis=(-1,-2))
    
    if test:
        print(lattice_vectors.shape, r_vectors.shape)
        print(e_e_Rs0.shape, e_e_Rs1.shape, r_term.shape)
        
    return average_potential

# unit test
k, rcut = 4.15826182047647, 1.7129888292804627
compute_potential(walkers, charges, k, rcut)

[-4 -3 -2 -1  0  1  2  3  4]
728 728
[10 10 10]
(1000, 3)
[-4 -3 -2 -1  0  1  2  3  4]
728 728
232 232
[[[1.47757581e-05]]]
r_term: [[[6545.70843667]]]
self_interation [[[-21.11443204]]]
constant  -1.6351918754903916


<ipython-input-142-cf91a04ba648>:11: RuntimeWarning: divide by zero encountered in true_divide
  e_e_Rs0 = q_q * np.tril(erfc(kappa * e_e_distances) / e_e_distances, k=-1)


array([6522.95882752])